In [3]:
# Import all the necessary libraries

import re
import sys
import itertools

import numpy as np
import pandas as pd

from random import choice
from bs4 import BeautifulSoup
from datetime import datetime as dt
from urllib.request import Request, urlopen


%matplotlib inline


In [6]:
# url = 'https://www.wettportal.com/quotenvergleich/surebets/'

url = 'https://www.wettportal.com/lib/ajax/getDatagridContent.php?lang=de&partner=wettportal&contenttype=surebets&offset=00&pagesize=82&period=all'

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
url = urlopen(req)
content = url.read()

soup = BeautifulSoup(content, 'lxml')


In [7]:
soup

<html><body><article class="container intro">
<div class="frame">
<div class="gutter"><p>Mit unserem Surebet Tool filtern wir minutenaktuell Surebets durch Vergleich der Wettquoten aller Wettanbieter für euch heraus. Bei Surebets wird auf alle möglichen Ausgänge gesetzt und trotzdem ein Profit erzielt, da die Buchmacher in ihren Wahrscheinlichkeitsberechnungen zum Teil weit auseinanderliegen. Mit einem Klick auf unseren Surebet Rechner übernehmt ihr die angegebenen Quoten, tragt nur noch euren Wetteinsatz ein und der Rechner zeigt euch, wie das Geld verteilt werden muss um eure Surebet gewinnen.</p> <p>Das Wetten auf <strong>Surebets</strong> oder auch Arbitrage Wetten funktioniert nur durch Einsätze bei mehreren Anbietern, die Wettquoten werden relativ schnell angeglichen und es empfiehlt sich, schon vorher Wettkonten bei den Topbuchmachern eröffnet zu haben und eine gewisse Geldmenge für euren Einsatz deponiert zu haben.</p> <p>Unser <strong>Surebet Tool</strong> berechnet zur Zeit d

In [8]:
divs = []
sports = []
country = []
leagues = []

def search_the_bet_divs():
    for div in soup.findAll('article', { "class" : "container surebets" }):
        #print(div)
        
        txt = str(div.findAll('h2'))

        re1='.*?'	# Non-greedy match on filler
        re2='(?:[a-z][a-z0-9_]*)'	# Uninteresting: var
        re3='.*?'	# Non-greedy match on filler
        re4='(?:[a-z][a-z0-9_]*)'	# Uninteresting: var
        re5='.*?'	# Non-greedy match on filler
        re6='((?:[a-z][a-z0-9_]*))'	# Variable Name 1

        rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
        m = rg.search(txt)
        if m:
            var1=m.group(1)
            print("("+var1+")"+"\n")
        
        divs.append(div)
    return divs


def search_the_bet_integer(text):
    for div in text.findAll('div', { "class" : 'left-col'}): #"span-class" : "count_all_events"}):
        t = str(div)
        re1='.*?'	# Non-greedy match on filler
        re2='\\d+'	# Uninteresting: int
        re3='.*?'	# Non-greedy match on filler
        re4='\\d+'	# Uninteresting: int
        re5='.*?'	# Non-greedy match on filler
        re6='(\\d+)'	# Integer Number 1

        rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
        m = rg.search(t)
        if m:
            int1=m.group(1)
            return(int(int1))

        
def get_the_profit_values(txt):
        
    re1='.*?'	# Non-greedy match on filler
    re2='([+-]?\\d*\\.\\d+)(?![-+0-9\\.])'	# Float 1

    rg = re.compile(re1+re2,re.IGNORECASE|re.DOTALL)
    m = rg.search(txt)
    if m:
        float1=m.group(1)
        return "("+float1+")"+"\n"

    
    
    
def get_date_of_the_event(txt):
    re1='.*?'	# Non-greedy match on filler
    re2='(20\\.06\\.2018)'	# DDMMYYYY 1

    rg = re.compile(re1+re2,re.IGNORECASE|re.DOTALL)
    m = rg.search(txt)
    if m:
        ddmmyyyy1=m.group(1)
        return ddmmyyyy1

    
    
#divs = search_the_bet_divs()
maxbet = search_the_bet_integer(soup)


print(maxbet)
print(type(maxbet))
print(type(soup))

45
<class 'int'>
<class 'bs4.BeautifulSoup'>


In [9]:

def random_useragent(url):

        HEADERS = {
                        'User-Agent': 'Mozilla/5.0',
                        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                        'Accept-Encoding': 'gzip, deflate',
                        'Connection': 'close',
                        'DNT': '1'
                }


        UAS = [
                        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.97 Safari/537.11',
                        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:17.0) Gecko/20100101 Firefox/17.0',
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/536.26.17 (KHTML, like Gecko) Version/6.0.2 Safari/536.26.17',
                        'Mozilla/5.0 (Linux; U; Android 2.2; fr-fr; Desire_A8181 Build/FRF91) App3leWebKit/53.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
                        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; FunWebProducts; .NET CLR 1.1.4322; PeoplePal 6.2)',
                        'Mozilla/5.0 (Windows NT 5.1; rv:13.0) Gecko/20100101 Firefox/13.0.1',
                        'Opera/9.80 (Windows NT 5.1; U; en) Presto/2.10.289 Version/12.01',
                        'Mozilla/5.0 (Windows NT 5.1; rv:5.0.1) Gecko/20100101 Firefox/5.0.1',
                        'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) ; .NET CLR 3.5.30729)'
                ]

        req = Request(url)
        HEADERS['User-Agent'] = choice(UAS)
        req.add_header('User-Agent', HEADERS)

        response = urlopen(req)
        html = response.read()

        return html

In [10]:
url = str('https://www.wettportal.com/lib/ajax/getDatagridContent.php?lang=de&partner=wettportal&contenttype=surebets&offset=00&pagesize='+ str(maxbet) + '&period=all')
print(url)

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
url = urlopen(req)
content = url.read()


# function to remove the whitespaces from a string - here the name of the event
def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']


#content = random_useragent(url)

soup = BeautifulSoup(content, 'lxml')

result = []


for div in soup.findAll('article', { "class" : "container surebets" }):
    
    #print(div)
    
    # get the time of the event
    eventtime = div.find('time').get_text()
        
    # get the event (teams, sport, etc.)
    l = div.find('span').get_text().split("|")
    l = strip_list_noempty(l)
    
    
    sport = l[0]
    type1 = l[1]
    type2 = l[2]
    
    # get the profitfactor of the event
    p = div.find('td', {"class" : "ralign"})
    print(type(p))
    # get the teames/ players/ athletes
    
    # put it all together for one line of data
    data = [eventtime, sport, type1, type2]
    result.append(data)
    continue
    
# check it line by line    
for i in result:
    print(i)
    break
    
    
    
    
"""
    for element in div.findAll("th", { "itemprop" : "startDate"}):
        result.append(get_time_of_the_event(element))
        print(result)
        print('#################################')
    
    # get the profit of the bet
    for element in div.findAll("span", { "class" : "hilite-red"}):
        result.append(element.string)
    
    #result.append(div.text)
    
    #for i in soup.findAll('span', { "class" : "hilite-redprint"}):
    #    print(i)
    #    r = get_the_profit_values(str(i))
    #    print(r)
    #print(result)
    break
    #result.append(get_the_profit_values(str(div)))

print(result)
print(len(result))

"""

https://www.wettportal.com/lib/ajax/getDatagridContent.php?lang=de&partner=wettportal&contenttype=surebets&offset=00&pagesize=45&period=all
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<c

'\n    for element in div.findAll("th", { "itemprop" : "startDate"}):\n        result.append(get_time_of_the_event(element))\n        print(result)\n        print(\'#################################\')\n    \n    # get the profit of the bet\n    for element in div.findAll("span", { "class" : "hilite-red"}):\n        result.append(element.string)\n    \n    #result.append(div.text)\n    \n    #for i in soup.findAll(\'span\', { "class" : "hilite-redprint"}):\n    #    print(i)\n    #    r = get_the_profit_values(str(i))\n    #    print(r)\n    #print(result)\n    break\n    #result.append(get_the_profit_values(str(div)))\n\nprint(result)\nprint(len(result))\n\n'

In [10]:
result.findAll('td', class_='ralign')

AttributeError: 'list' object has no attribute 'findAll'

In [ ]:
# next try

from selenium import webdriver as driver

from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

binary = FirefoxBinary('/home/robinarthur/Downloads/geckodriver/')
browser = driver.Firefox(firefox_binary=binary)




#browser = driver.Firefox()
p = browser.get("https://www.wettportal.com/quotenvergleich/surebets/")
print(p)
assert "Stack Overflow - Wikipedia" in browser.title